In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
my_key="sk-proj-HAJg2iDp8Tlk6XkkiO1RT3BlbkFJeyAkYh46OnNIaAg7igfC"

In [4]:
llm=ChatOpenAI(openai_api_key=my_key,model_name="gpt-3.5-turbo", temperature=0.8)

C:\Users\Admin\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002878E61BCD0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002878E623E50>, temperature=0.8, openai_api_key='sk-proj-HAJg2iDp8Tlk6XkkiO1RT3BlbkFJeyAkYh46OnNIaAg7igfC', openai_proxy='')

In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [10]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Admin\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
#seequential chain

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [16]:
with open('data.txt', 'r') as file:
    TEXT = file.read()

In [17]:
TEXT

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [21]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [22]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Admin\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [24]:
response.keys()

dict_keys(['text', 'number', 'subject', 'tone', 'response_json', 'quiz', 'review'])

In [25]:
response['quiz']

'{\n    "1": {\n        "mcq": "What is the scientific study of life called?",\n        "options": {\n            "a": "Chemistry",\n            "b": "Biology",\n            "c": "Physics",\n            "d": "Geology"\n        },\n        "correct": "b"\n    },\n    "2": {\n        "mcq": "Which theme in biology explains the unity and diversity of life?",\n        "options": {\n            "a": "Evolution",\n            "b": "Photosynthesis",\n            "c": "Reproduction",\n            "d": "Metabolism"\n        },\n        "correct": "a"\n    },\n    "3": {\n        "mcq": "What is the basic unit of life that all organisms are made up of?",\n        "options": {\n            "a": "Atoms",\n            "b": "Molecules",\n            "c": "Cells",\n            "d": "Tissues"\n        },\n        "correct": "c"\n    },\n    "4": {\n        "mcq": "What process allows organisms to move, grow, and reproduce?",\n        "options": {\n            "a": "Photosynthesis",\n            "b": "

In [27]:
quiz=response.get("quiz")

In [28]:
quiz=json.loads(quiz)

In [29]:
quiz

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Photosynthesis',
   'c': 'Reproduction',
   'd': 'Metabolism'},
  'correct': 'a'},
 '3': {'mcq': 'What is the basic unit of life that all organisms are made up of?',
  'options': {'a': 'Atoms', 'b': 'Molecules', 'c': 'Cells', 'd': 'Tissues'},
  'correct': 'c'},
 '4': {'mcq': 'What process allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Photosynthesis',
   'b': 'Respiration',
   'c': 'Digestion',
   'd': 'Energy processing'},
  'correct': 'd'},
 '5': {'mcq': 'What is the method that biologists use to make observations, pose questions, and form conclusions?',
  'options': {'a': 'Scientific method',
   'b': 'Guessing method',
   'c': 'Magic method',
   'd': 'Trial and error metho

In [32]:
#making table for q&a response

In [30]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [31]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Photosynthesis | c: Reproduction | d: Metabolism',
  'Correct': 'a'},
 {'MCQ': 'What is the basic unit of life that all organisms are made up of?',
  'Choices': 'a: Atoms | b: Molecules | c: Cells | d: Tissues',
  'Correct': 'c'},
 {'MCQ': 'What process allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Photosynthesis | b: Respiration | c: Digestion | d: Energy processing',
  'Correct': 'd'},
 {'MCQ': 'What is the method that biologists use to make observations, pose questions, and form conclusions?',
  'Choices': 'a: Scientific method | b: Guessing method | c: Magic method | d: Trial and error method',
  'Correct': 'a'}]

In [33]:
quiz=pd.DataFrame(quiz_table_data)

In [34]:
quiz

,MCQ,Choices,Correct
0,What is the scientific study of life called?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Photosynthesis | c: Reproduc...,a
2,What is the basic unit of life that all organi...,a: Atoms | b: Molecules | c: Cells | d: Tissues,c
3,"What process allows organisms to move, grow, a...",a: Photosynthesis | b: Respiration | c: Digest...,d
4,What is the method that biologists use to make...,a: Scientific method | b: Guessing method | c:...,a
